In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data

from module.dataset import ModelNet40
from module.utils import *

import os, sys
from collections import OrderedDict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
class BatchNorm(nn.Module):
    '''
        Perform batch normalization.
        Input: A tensor of size (N, M, feature_dim), or (N, feature_dim, M) (available when feature_dim != M), 
                or (N, feature_dim)
        Output: A tensor of the same size as input.
    '''
    def __init__(self, feature_dim):
        super(BatchNorm, self).__init__()
        self.feature_dim = feature_dim
        self.batchnorm = nn.BatchNorm1d(feature_dim)
        self.permute = Permute((0, 2, 1))

    def forward(self, x):
        if (len(x.shape) == 3) and (x.shape[-1] == self.feature_dim):
            return self.permute(self.batchnorm(self.permute(x)))
        else:
            return self.batchnorm(x)

In [23]:
class Permute(nn.Module):
    def __init__(self, param):
        super(Permute, self).__init__()
        self.param = param

    def forward(self, x):
        return x.permute(self.param)

In [24]:
class MLP(nn.Module):
    def __init__(self, hidden_size, batchnorm = True, last_activation = True):
        super(MLP, self).__init__()
        q = []
        for i in range(len(hidden_size)-1):
            in_dim = hidden_size[i]
            out_dim = hidden_size[i+1]
            q.append(("Linear_%d" % i, nn.Linear(in_dim, out_dim)))
            if (i < len(hidden_size) - 2) or ((i == len(hidden_size) - 2) and (last_activation)):
                if (batchnorm):
                    q.append(("Batchnorm_%d" % i, BatchNorm(out_dim)))
                q.append(("ReLU_%d" % i, nn.ReLU(inplace=True)))
        self.mlp = nn.Sequential(OrderedDict(q))

    def forward(self, x):
        return self.mlp(x)

In [25]:
class MaxPooling(nn.Module):
    def __init__(self):
        super(MaxPooling, self).__init__()

    def forward(self, x, dim=1, keepdim = False):
        res, _ = torch.max(x, dim=dim, keepdim = keepdim)
        return res

In [26]:
class TNet(nn.Module):
    def __init__(self, nfeat):
        super(TNet, self).__init__()
        self.nfeat = nfeat
        self.tnet = nn.Sequential(MLP((nfeat, 64, 128, 1024)), MaxPooling(), 
                                  BatchNorm(1024), MLP((1024, 512, 256, nfeat*nfeat)))
        
    def forward(self, x):
        batch_size = x.shape[0]
        return self.tnet(x).view(batch_size, self.nfeat, self.nfeat)

In [27]:
class PointNet(nn.Module):
    def __init__(self, nfeat, nclass, dropout = 0):
        super(PointNet, self).__init__()

        self.input_transform = TNet(nfeat)
        self.mlp1 = nn.Sequential(BatchNorm(3), MLP((nfeat, 64, 64)))
        self.feature_transform = TNet(64)
        self.mlp2 = nn.Sequential(BatchNorm(64), MLP((64, 64, 128, 1024)))
        self.maxpooling = MaxPooling()
        self.mlp3 = nn.Sequential(BatchNorm(1024), MLP((1024, 512, 256)), nn.Dropout(dropout), nn.Linear(256, nclass))
        
        self.eye64 = torch.eye(64).to(device)

    def forward(self, xs):
        batch_size = xs.shape[0]
        
        transform = self.input_transform(xs)
        xs = torch.stack([torch.mm(xs[i],transform[i]) for i in range(batch_size)])
        xs = self.mlp1(xs)
        
        transform = self.feature_transform(xs)
        xs = torch.stack([torch.mm(xs[i],transform[i]) for i in range(batch_size)])
        xs = self.mlp2(xs)
        
        xs = self.mlp3(self.maxpooling(xs))
        
        if (self.training):
            transform_transpose = transform.transpose(1, 2)
            tmp = torch.stack([torch.mm(transform[i], transform_transpose[i]) for i in range(batch_size)])
            L_reg = ((tmp - self.eye64) ** 2).sum() / batch_size
            
        return (F.log_softmax(xs, dim=1), L_reg) if self.training else F.log_softmax(xs, dim=1)

In [33]:
lr = 0.001
num_points = 128
save_name = "PointNet.pt"
#batch_size = 512
batch_size = 16

########### loading data ###########

train_data = ModelNet40(num_points)
test_data = ModelNet40(num_points, 'test')

train_size = int(0.9 * len(train_data))
valid_size = len(train_data) - train_size
train_data, valid_data = Data.random_split(train_data, [train_size, valid_size])
valid_data.partition = 'valid'
train_data.partition = 'train'

print("train data size: ", len(train_data))
print("valid data size: ", len(valid_data))
print("test data size: ", len(test_data))

def collate_fn(batch):
    Xs = torch.stack([X for X, _ in batch])
    Ys = torch.tensor([Y for _, Y in batch], dtype = torch.long)
    return Xs, Ys

train_iter  = Data.DataLoader(train_data, shuffle = True, batch_size = batch_size, collate_fn = collate_fn)
valid_iter = Data.DataLoader(valid_data, batch_size = batch_size, collate_fn = collate_fn)
test_iter = Data.DataLoader(test_data, batch_size = batch_size, collate_fn = collate_fn)

train data size:  8856
valid data size:  984
test data size:  2468


In [34]:
############### loading model ####################

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = PointNet(nfeat=3, nclass=40, dropout=0.3)
net.to(device)
print(net)

PointNet(
  (input_transform): TNet(
    (tnet): Sequential(
      (0): MLP(
        (mlp): Sequential(
          (Linear_0): Linear(in_features=3, out_features=64, bias=True)
          (Batchnorm_0): BatchNorm(
            (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (permute): Permute()
          )
          (ReLU_0): ReLU(inplace=True)
          (Linear_1): Linear(in_features=64, out_features=128, bias=True)
          (Batchnorm_1): BatchNorm(
            (batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (permute): Permute()
          )
          (ReLU_1): ReLU(inplace=True)
          (Linear_2): Linear(in_features=128, out_features=1024, bias=True)
          (Batchnorm_2): BatchNorm(
            (batchnorm): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (permute): Permute()
          )
          (ReLU_2): ReLU(inplace=Tr

In [35]:
############### training #########################

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr, weight_decay = 0.0001)
loss = nn.NLLLoss()

def adjust_lr(optimizer, decay_rate=0.95):
    for param_group in optimizer.param_groups:
        param_group['lr'] *= decay_rate

retrain = True
if os.path.exists(save_name):
    print("Model parameters have already been trained before. Retrain ? (y/n)")
    ans = input()
    if (ans == 'y'):
        checkpoint = torch.load(save_name, map_location = device)
        net.load_state_dict(checkpoint["net"])
        optimizer.load_state_dict(checkpoint["optimizer"])
        for g in optimizer.param_groups:
            g['lr'] = lr

train_model(train_iter, valid_iter, net, loss, optimizer, device = device, max_epochs = int(1000/(batch_size/64)), 
            adjust_lr = adjust_lr, early_stop = EarlyStop(patience = 20, save_name = save_name))
    

############### testing ##########################

loss, acc = evaluate_model(test_iter, net, loss)
print('test acc = %.6f' % (acc))


training on cuda


100%|██████████| 554/554 [00:15<00:00, 35.30it/s]


epoch 1 / 4000, train loss 3.3343 (acc 0.119015), valid loss 3.2928 (acc 0.166667)


100%|██████████| 554/554 [00:15<00:00, 35.47it/s]


epoch 2 / 4000, train loss 2.9125 (acc 0.219851), valid loss 2.9936 (acc 0.217480)


...

epoch 54 / 4000, train loss 0.4349 (acc 0.861337), valid loss 1.5773 (acc 0.816057)
EarlyStopping counter: 18 out of 20


100%|██████████| 554/554 [00:15<00:00, 35.25it/s]


epoch 55 / 4000, train loss 0.4174 (acc 0.866983), valid loss 18.8756 (acc 0.785569)
EarlyStopping counter: 19 out of 20


100%|██████████| 554/554 [00:15<00:00, 35.66it/s]


epoch 56 / 4000, train loss 0.4092 (acc 0.871500), valid loss 1.1155 (acc 0.843496)
EarlyStopping counter: 20 out of 20
test acc = 0.825770
